In [ ]:
import pandas as pd
import json
import glob
import matplotlib.pyplot as plt
import numpy as np

import sklearn
import sklearn.metrics
import matplotlib
import scipy

In [ ]:
fis = glob.glob("experiments/cms-gnn-skipconn-cb3939ed/logs_*.json")

In [ ]:
ds = []
for fi in fis:
    d = json.load(open(fi, 'r'))
    l2_split = d.pop("l2_split")
    for i in range(len(l2_split)):
        d["l2_{}".format(i)] = l2_split[i]
    ds.append(d)

In [ ]:
df = pd.DataFrame.from_dict(ds)
df = df.sort_values('epoch')

In [ ]:
df

In [ ]:
plt.plot(df['l1'].values)

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(df['l1'], df['l2'], marker=".")
plt.xlabel("total cls loss")
plt.ylabel("total reg loss")

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(df['l1'], df['l3'], marker=".")
plt.xlabel("total cls loss")
plt.ylabel("total chg loss")

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(df['l2_0'], df['l2_1'], marker=".")

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(df['l2_0'], df['l2_2'], marker=".")

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(df['l2_0'], df['l2_3'], marker=".")

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(df['l2_0'], df['l2_4'], marker=".")

In [ ]:
for i in range(5):
    plt.plot(df["l2_{}".format(i)].values, label="{}".format(i))
plt.legend()
#plt.yscale("log")

In [ ]:
dd = np.load("experiments/cms-gnn-skipconn-cb3939ed/pred.npz")

In [ ]:
list(dd.keys())

In [ ]:
X = dd["X"]
ygen = dd["ygen"]
ycand = dd["ycand"]
ypred = dd["ypred"]
ypred_raw = dd["ypred_raw"]

X_f = X.reshape((X.shape[0]*X.shape[1], X.shape[2]))
ygen_f = ygen.reshape((ygen.shape[0]*ygen.shape[1], ygen.shape[2]))
ycand_f = ycand.reshape((ycand.shape[0]*ycand.shape[1], ycand.shape[2]))
ypred_f = ypred.reshape((ypred.shape[0]*ypred.shape[1], ypred.shape[2]))
ypred_raw_f = ypred_raw.reshape((ypred_raw.shape[0]*ypred_raw.shape[1], ypred_raw.shape[2]))
ypred_raw_f_sm = scipy.special.softmax(ypred_raw_f, axis=-1)

In [ ]:
ypred_f[(ypred_f[:, 0]==3) & (ypred_raw_f_sm[:, 3]<0.6), 0] = 0

In [ ]:
c = np.unique(ycand_f[:, 0], return_counts=True)
plt.pie(c[1], labels=c[0]);

In [ ]:
plt.hist(
    np.sum(X[:, :, 0]!=0, axis=1),
    bins=100
)

In [ ]:
#       NONE = 0,
#       TRACK = 1,
#       PS1 = 2,
#       PS2 = 3,
#       ECAL = 4,
#       HCAL = 5,
#       GSF = 6,
#       BREM = 7,
#       HFEM = 8,
#       HFHAD = 9,
#       SC = 10,
#       HO = 11,
#       HGCAL = 12,

In [ ]:
# [0, 211, 130, 1, 2, 22, 11, 13]

In [ ]:
cm = sklearn.metrics.confusion_matrix(ycand_f[:, 0], ypred_f[:, 0], labels=range(8))

In [ ]:
plt.imshow(cm, cmap="Blues", norm=matplotlib.colors.LogNorm())
plt.colorbar()

In [ ]:
pid = 4
m0 = ygen_f[:, 0] == pid
plt.hist(ypred_raw_f_sm[m0, pid], bins=np.linspace(0,1,41), histtype="step"); 
plt.hist(ypred_raw_f_sm[~m0, pid], bins=np.linspace(0,1,41), histtype="step"); 
plt.yscale("log")

In [ ]:
fpr, tpr, thresh = sklearn.metrics.roc_curve(m0, ypred_raw_f_sm[:, pid])

In [ ]:
plt.plot(thresh, fpr)
plt.plot(thresh, tpr)
plt.yscale("log")
plt.xlim(0,1)

In [ ]:
msk = (X_f[:, 0]==8) & (ygen_f[:, 0]!=0) & (ycand_f[:, 0]!=0)

In [ ]:
plt.hist(ygen_f[msk, 2], bins=np.linspace(0, 5, 41), histtype="step");
plt.hist(ycand_f[msk, 2], bins=np.linspace(0, 5, 41), histtype="step");
plt.hist(ypred_f[msk, 2], bins=np.linspace(0, 5, 41), histtype="step");

In [ ]:
c = np.unique(ygen_f[msk, 0], return_counts=True)
plt.pie(c[1], labels=c[0]);

In [ ]:
c = np.unique(ycand_f[msk, 0], return_counts=True)
plt.pie(c[1], labels=c[0]);

In [ ]:
plt.hist(ygen_f[msk, 3], bins=np.linspace(-7, 7, 41), histtype="step");
plt.hist(ycand_f[msk, 3], bins=np.linspace(-7, 7, 41), histtype="step");

In [ ]:
plt.figure(figsize=(5,5))

for pid in [1,2,3,4]:
    plt.figure(figsize=(5,5))
    plt.title("pid={}".format(pid))
    
    a = np.sum(ygen[:, :, 0]==pid, axis=1)
    b = np.sum(ycand[:, :, 0]==pid, axis=1)
    c = np.sum(ypred[:, :, 0]==pid, axis=1)
    
    minval = min([np.min(a), np.min(b), np.min(c)])
    maxval = max([np.max(a), np.max(b), np.max(c)])
    
    plt.plot([minval, maxval], [minval, maxval], ls="--", color="black")
    plt.scatter(
        b,
        c,
        marker=".", alpha=0.5
    )
    
#     plt.scatter(
#         a,
#         b,
#         marker=".", alpha=0.5
#     )

#     plt.scatter(
#         a,
#         c,
#         marker=".", alpha=0.5
#     )
    
    plt.xlim(minval, maxval)
    plt.ylim(minval, maxval)

In [ ]:
pid = 2
msk = (ygen_f[:, 0]==pid) & (ycand_f[:, 0]==pid) & (ypred_f[:, 0]==pid)

In [ ]:
b = np.linspace(0, 10, 100)
plt.hist(ygen_f[msk, 2], bins=b, histtype="step", lw=2, label="gen");
plt.hist(ycand_f[msk, 2], bins=b, histtype="step", lw=2, label="cand");
plt.hist(ypred_f[msk, 2], bins=b, histtype="step", lw=2, label="pred");
plt.legend(loc="best")

In [ ]:
b = np.linspace(-5, 5, 100)
plt.hist(ygen_f[msk, 3], bins=b, histtype="step", lw=2, label="gen");
plt.hist(ycand_f[msk, 3], bins=b, histtype="step", lw=2, label="cand");
plt.hist(ypred_f[msk, 3], bins=b, histtype="step", lw=2, label="pred");
plt.legend(loc="best")

In [ ]:
b = np.linspace(0, 50, 100)
plt.hist(ygen_f[msk, 6], bins=b, histtype="step", lw=2);
plt.hist(ycand_f[msk, 6], bins=b, histtype="step", lw=2);
plt.hist(ypred_f[msk, 6], bins=b, histtype="step", lw=2);
plt.legend(loc="best")

In [ ]:
for pid in [1,2,3,4]:
    
    msk = (ygen_f[:, 0]==pid) & (ycand_f[:, 0]==pid) & (ypred_f[:, 0]==pid)
    
    for var in [2,3,4,5,6]:
        a = ygen_f[msk, var]
        b = ycand_f[msk, var]
        c = ypred_f[msk, var]

        plt.figure(figsize=(4,4))
        plt.title("pid={} var={}".format(pid, var))
        r1 = (b-a)/a
        r2 = (c-a)/a
        
        msk1 = np.abs(r1)<10
        mean1 = np.mean(r1[msk1])
        std1 = np.std(r1[msk1])
    
        msk2 = np.abs(r2)<10
        mean2 = np.mean(r2[msk2])
        std2 = np.std(r2[msk2])
        
        plt.hist(r1, bins=np.linspace(-2, 2, 100),
            histtype="step", label="PF m={:.2f} s={:.2f}".format(mean1, std1));
        plt.hist((c-a)/a, bins=np.linspace(-2, 2, 100),
            histtype="step", label="MLPF m={:.2f} s={:.2f}".format(mean2, std2));

        plt.legend(loc="best")